In [5]:
import tensorflow as tf
import numpy as np

#sess = tf.InteractiveSession()

In [3]:
num_puntos = 224 * 224 * 3
num_puntos

150528

In [6]:
conjunto_puntos = []
for i in xrange(num_puntos):
   if np.random.random() > 0.5:
     conjunto_puntos.append([np.random.normal(0.0, 0.9), np.random.normal(0.0, 0.9)])
   else:
     conjunto_puntos.append([np.random.normal(3.0, 0.5), np.random.normal(1.0, 0.5)])

## Real time Recognition Layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 150528])
y_ = tf.placeholder(tf.float32, shape=[None, 6])

W = tf.Variable(tf.zeros([150528,6]))
b = tf.Variable(tf.zeros([6]))

#sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)


# conv1
with tf.variable_scope('conv1') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[7, 7, 96],
                                         stddev=5e-2,
                                         wd=0.0)
    conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_on_cpu('biases', [96], tf.constant_initializer(0.0))
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope.name)
    _activation_summary(conv1)

# pool1
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1],
                         padding='SAME', name='pool1')
# norm1
norm1 = tf.nn.relu(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')

# conv2
with tf.variable_scope('conv2') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[5, 5, 96, 256],
                                         stddev=5e-2,
                                         wd=0.0)
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_on_cpu('biases', [256], tf.constant_initializer(0.1))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)
    _activation_summary(conv2)

# norm2
norm2 = tf.nn.relu(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
# pool2
pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1],
                         strides=[1, 1, 1, 1], padding='SAME', name='pool2')


# conv3
with tf.variable_scope('conv3') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[3, 3, 256, 512],
                                         stddev=5e-2,
                                         wd=0.0)
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_on_cpu('biases', [512], tf.constant_initializer(0.1))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)
    _activation_summary(conv3)
    
# norm3
norm3 = tf.nn.relu(conv3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm3')

# conv4
with tf.variable_scope('conv4') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[3, 3, 512, 512],
                                         stddev=5e-2,
                                         wd=0.0)
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_on_cpu('biases', [512], tf.constant_initializer(0.1))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)
    _activation_summary(conv4)
    
# norm4
norm4 = tf.nn.relu(conv4, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm4')    

# conv5
with tf.variable_scope('conv5') as scope:
    kernel = _variable_with_weight_decay('weights',
                                         shape=[3, 3, 512, 512],
                                         stddev=5e-2,
                                         wd=0.0)
    conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
    biases = _variable_on_cpu('biases', [512], tf.constant_initializer(0.1))
    bias = tf.nn.bias_add(conv, biases)
    conv2 = tf.nn.relu(bias, name=scope.name)
    _activation_summary(conv5)

# pool5
pool5 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                         strides=[1, 1, 1, 1], padding='SAME', name='pool5')


#Flatten
h_pool5_flat = tf.reshape(pool5, [-1, 12801])

# fully connected layer 6
W_fc6 = weight_variable([12801, 4048])
b_fc6 = bias_variable([4048])

#Relu 6
h_fc6 = tf.nn.relu(tf.matmul(h_pool5_flat, W_fc6) + b_fc6)

# Dropout 6
keep_prob = tf.placeholder(tf.float32)
h_fc6_drop = tf.nn.dropout(h_fc6, keep_prob)

#fully connected layer 7
W_fc7 = weight_variable([4049, 4048])
b_fc7 = bias_variable([4048])

#Flatten7
#h_pool5_flat = tf.reshape(pool5, [-1, 12801])

#Relu 7
h_fc7 = tf.nn.relu(tf.matmul(h_pool5_flat, W_fc7) + b_fc7)

# Dropout 7
keep_prob = tf.placeholder(tf.float32)
h_fc7_drop = tf.nn.dropout(h_fc7, keep_prob)


#fully connected layer 8
W_fc8 = weight_variable([4049, 6])
b_fc8 = bias_variable([6])

y_conv=tf.nn.softmax(tf.matmul(h_fc7_drop, W_fc8) + b_fc8)

# Train and Evaluate the Model

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

for i in range(20000):
  #batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))